# Set up and installation of modules

In [ ]:
# Uncomment as needed

#!pip install scikit-learn
#!pip install tensorflow
#!pip install gymnasium
#!pip install pygame
#!pip install stable-baselines3

# Principal Component Analysis

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
ones = np.loadtxt("ones.txt")
zeros = np.loadtxt("zeros.txt")

time_stamps = ones[:,0]
data_ones = ones[:,1:]
data_zeros = zeros[:,1:]

num_ones_samples = data_ones.shape[1]
num_zeros_samples = data_zeros.shape[1]

plt.plot(t, data_ones[:,2])
plt.plot(t, data_zeros[:,2])
plt.xlabel("time (s)")
plt.ylabel("signal (V)")

In [ ]:
X = np.hstack([data_ones, data_zeros]).T
print(X.shape)

colors = ['r'] * num_ones_samples + ['b'] * num_zeros_samples
pca = PCA(n_components=2)
pca.fit(X)

Y = pca.transform(X)
print(f"The first component explains {pca.explained_variance_ratio_[0]*100}% of the variance, and the second explains {pca.explained_variance_ratio_[1]*100}%")

fig, ax = plt.subplots(2)
ax[0].scatter(Y[:, 0], Y[:, 1], c=colors, edgecolor="w")
ax[0].set_xlabel("Principal component 1")
ax[0].set_ylabel("Principal component 2")

ax[1].plot(t, pca.components_[0], label="principal component 0")
ax[1].plot(t, pca.components_[1], label="principal component 1")
ax[1].set_xlabel("time (s)")
ax[1].set_ylabel("principal component")
ax[1].legend()

fig.tight_layout()

# Supervised learning with a feedforward neural network

In [ ]:
from tensorflow import keras
from keras.utils import to_categorical

This cell will download the MNIST dataset if it hasn't already been downloaded :)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Show one image
plt.imshow(x_train[0], cmap='gray')
print(f"This image is a {y_train[0]}")

# Turn the answers into one-hot vectors
y_train = to_categorical(y_train, num_classes=10)
print(f"This corresponds to the one-hot vector: ", y_train[0])

# Reshape each image from a 28x28 2D array into a 1D array of size 28*28
x_train = np.reshape(x_train, (-1, 28*28))
# Normalize the inputs to lie between 0 and 1
x_train = x_train/256.0

# Reshape the test set
x_test = np.reshape(x_test, (-1,28*28))/256
y_test = to_categorical(y_test, num_classes=10)

### Build a small neural network

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(40, input_shape=(28*28,)))
model.add(keras.layers.Dense(10))

# Finalize the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train it!
history = model.fit(x_train, y_train, batch_size=32, epochs=10)

### Test individual cases

In [ ]:
check = np.random.randint(0, len(x_test))

plt.imshow(np.reshape(x_test[check], (28,28)))
answer = np.argmax(model.predict(np.reshape(x_test[check], (1, 28*28))))
print(f"This is a(n) {answer}")

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(test_accuracy)

## CNN

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10))

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

# Unsupervised learning with a feedforward neural network

https://keras.io/examples/generative/conditional_gan/

# Reinforcement learning

In [ ]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode=None)

observation, info = env.reset()
for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)

In [ ]:
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()
while True:
    action, _states = model.predict(observation)
    observation, reward, terminated, truncated, info = env.step(action)